In [2]:
!pip install bert-score --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 109.9 MB/s eta 0:00:00


In [3]:
# Importing paths
import pandas as pd
from bert_score import score as bert_score
import torch

In [4]:
!wget https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
!tar -xf abo-images-small.tar

--2025-05-14 18:24:13--  https://amazon-berkeley-objects.s3.amazonaws.com/archives/abo-images-small.tar
Resolving amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)... 3.5.25.225, 52.217.162.9, 52.217.67.164, ...
Connecting to amazon-berkeley-objects.s3.amazonaws.com (amazon-berkeley-objects.s3.amazonaws.com)|3.5.25.225|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3253381120 (3.0G) [application/x-tar]
Saving to: ‘abo-images-small.tar’

abo-images-small.ta 100%[===================>]   3.03G  33.6MB/s    in 2m 6s   

2025-05-14 18:26:19 (24.7 MB/s) - ‘abo-images-small.tar’ saved [3253381120/3253381120]



### **Evaluating for QWEN7B**

In [5]:
# Set paths
CSV_PATH = "/content/drive/MyDrive/images/Results/results_QWEN7B.csv"
IMAGE_ROOT = "/content/images/small/"

# Load predictions CSV
df = pd.read_csv(CSV_PATH)

In [6]:
# Lowercase both predictions and ground truth for string matching
df['answer'] = df['answer'].astype(str).str.lower().str.strip()
df['generated_answer'] = df['generated_answer'].astype(str).str.lower().str.strip()

In [8]:
df.head(10)

,image_path,question,answer,generated_answer
0,a1/a1f37cc0.jpg,What is the pattern?,comforter,solid
1,77/772f6f1f.jpg,What brand is this?,206,ugg
2,1d/1d3c1567.jpg,What is the brand?,solimo,mtt
3,fb/fb57539f.jpg,Is it 3D?,yes,no
4,e0/e030ba61.jpg,What is the product type?,grocery,100% arabica coffee beans
5,49/49b1b22b.jpg,What type of case is it?,hard,optimal lift
6,1f/1f243ec0.jpg,What kind of shoe is it?,running,leone
7,50/50ce84a2.jpg,What type of wash?,handwash,liquid handwash
8,49/49b1b22b.jpg,What is the product's brand?,solimo,apple
9,0f/0fc9d6c7.jpg,What is the product type?,case,phone case


In [9]:
# String Matching Accuracy
df['is_match'] = df['answer'] == df['generated_answer']
string_match_accuracy = df['is_match'].mean()

In [10]:
# BERTScore
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

P, R, F1 = bert_score(candidates, references, lang='en', device='cuda' if torch.cuda.is_available() else 'cpu')
average_bertscore = F1.mean().item()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Display Results
print(f"String Matching Accuracy: {string_match_accuracy:.4f}")
print(f"Average BERTScore (F1):   {average_bertscore:.4f}")

String Matching Accuracy: 0.2710
Average BERTScore (F1):   0.9192


### **Evaluating for QWEN2B**

In [14]:
# Set paths
CSV_PATH = "/content/drive/MyDrive/images/Results/results_qwen2B.csv"

In [15]:
# Load predictions CSV
df = pd.read_csv(CSV_PATH)

In [16]:
# Lowercase both predictions and ground truth for string matching
df['answer'] = df['answer'].astype(str).str.lower().str.strip()
df['generated_answer'] = df['generated_answer'].astype(str).str.lower().str.strip()

In [17]:
df.head(5)

,image_path,question,answer,generated_answer
0,a1/a1f37cc0.jpg,What is the pattern?,comforter,dance
1,77/772f6f1f.jpg,What brand is this?,206,apple
2,1d/1d3c1567.jpg,What is the brand?,solimo,tangram
3,fb/fb57539f.jpg,Is it 3D?,yes,no
4,e0/e030ba61.jpg,What is the product type?,grocery,clothing


In [18]:
# String Matching Accuracy
df['is_match'] = df['answer'] == df['generated_answer']
string_match_accuracy = df['is_match'].mean()

In [19]:
# BERTScore
references = df['answer'].tolist()
candidates = df['generated_answer'].tolist()

P, R, F1 = bert_score(candidates, references, lang='en', device='cuda' if torch.cuda.is_available() else 'cpu')
average_bertscore = F1.mean().item()

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Display Results
print(f"String Matching Accuracy: {string_match_accuracy:.4f}")
print(f"Average BERTScore (F1):   {average_bertscore:.4f}")

String Matching Accuracy: 0.1032
Average BERTScore (F1):   0.9288
